In [ ]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import keras
from keras import regularizers,metrics 
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from random import shuffle
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from sklearn.metrics import confusion_matrix,plot_confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

# <h2>Dataset Preparation  ---- Preprocessing</h2>

In [ ]:
def create_dataset(path):
    dataset = pd.read_csv(path)
    print(dataset.shape)
    X=dataset.iloc[:,1:].to_numpy().astype("float32")
    y=dataset["label"]
    return X.reshape(X.shape[0],28,28,1),y

In [ ]:
X_train,y_train = create_dataset("../input/regularization-techniques/train.csv")
X_test,y_test   = create_dataset("../input/regularization-techniques/test.csv")

In [ ]:
num_classes = len(set(y_train)|set(y_test))+1
num_classes
epochs=20;

In [ ]:
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

In [ ]:
(X_train.shape)

In [ ]:
plt.imshow(np.array(X_train[20].reshape(28,28)))

# Model

In [ ]:
def Model(init_shape,regularizer):
    
    model = Sequential()
    keras.Input(
    shape=init_shape, batch_size=32)
    model.add(BatchNormalization())
    model.add(Conv2D(128,2,activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64,2,activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64,2,activation="relu"))   
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(32,activation="relu",kernel_regularizer=regularizer))
    model.add(Dense(num_classes,activation="softmax"))
    return model

In [ ]:
def evalu(model,X,y):
    preds=model.predict(X);
    preds=np.argmax(preds,axis=1)
    actual=np.argmax(y,axis=1)
    from mlxtend.plotting import plot_confusion_matrix
    cm = confusion_matrix(actual,preds)
    fig,ax=plot_confusion_matrix(conf_mat=cm,figsize=(12,12))
    plt.show()

# L1 regularisation

In [ ]:
regularizer=regularizers.l1(0.001)
model_l1 = Model((28,28,1),regularizer)
opt = keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.0001, nesterov = False)
model_l1.compile(loss="categorical_crossentropy",
                 optimizer=opt,
                 metrics = [metrics.categorical_accuracy],
                )
history_l1=model_l1.fit(X_train, y_train, batch_size = 32, epochs = epochs, validation_data = (X_test, y_test))

In [ ]:
evalu(model_l1,X_test,y_test)

# L2 Regularisation
* Adam optimizer
* lr=0.001
    


In [ ]:
regularizer=regularizers.l2(0.001)
model_l2 = Model((28,28,1),regularizer)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model_l2.compile(loss="categorical_crossentropy",
                 optimizer=opt,
                 metrics = [metrics.categorical_accuracy],
                )
history_l2 = model_l2.fit(X_train, y_train, batch_size = 32, epochs = epochs, validation_data = (X_test, y_test))

# Model with Dropout

In [ ]:
def Model_withDropout(init_shape):
    
    model = Sequential()
    keras.Input(
    shape=init_shape, batch_size=32)
    model.add(BatchNormalization())
    model.add(Conv2D(128,2,activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64,2,activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64,2,activation="relu"))   
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(32,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes,activation="softmax"))
    return model

In [ ]:
model_l1 = Model_withDropout(X_train[0].shape)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model_l1.compile(loss="categorical_crossentropy",
                 optimizer=opt,
                 metrics = [metrics.categorical_accuracy],
                )
history_l1=model_l1.fit(X_train, y_train, 
                        batch_size = 32, 
                        epochs = epochs, 
                        validation_data = (X_test, y_test))

# Model with DataAugmentation

In [ ]:
imageDatagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=(1/255.0),
)

In [ ]:
imageDatagen.fit(X_train)

In [ ]:
model_do = Model_withDropout(X_train[0].shape)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model_do.compile(loss="categorical_crossentropy",
                 optimizer=opt,
                 metrics = [metrics.categorical_accuracy],
                )
history_do=model_do.fit_generator(imageDatagen.flow(X_train, y_train, batch_size = 32), epochs = epochs, validation_data = (X_test, y_test))

# Early Stopping

In [ ]:
early_stopping = EarlyStopping(monitor="val_categorical_accuracy",patience=6)
model_es = Model_withDropout(X_train[0].shape)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model_es.compile(loss="categorical_crossentropy",
                 optimizer=opt,
                 metrics = [metrics.categorical_accuracy],
                )
history_es=model_es.fit(X_train, y_train, epochs = epochs, validation_data = (X_test, y_test),
                                 callbacks=[early_stopping])

# Classifiers with Ensemble methods

In [ ]:
def create_dataset(path):
    dataset = pd.read_csv(path)
    print(dataset.shape)
    X=dataset.iloc[:,1:].to_numpy().astype("float32")
    y=dataset["label"]
    return X,y

In [ ]:
X_train,y_train = create_dataset("../input/regularization-techniques/train.csv")
X_test,y_test   = create_dataset("../input/regularization-techniques/test.csv")

In [ ]:
from sklearn.ensemble import VotingClassifier,BaggingClassifier,RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression,LinearRegression,Ridge,Lasso,LassoLars,BayesianRidge
from sklearn.linear_model import MultiTaskElasticNet

In [ ]:
estimator=[]
estimator.append(("LR",LogisticRegression(multi_class="multinomial",max_iter=2000)))
estimator.append(("SVM",SVC(gamma="auto",probability=True)))
estimator.append(("DC",DecisionTreeClassifier()))

# 1.Hard Voting Classifier 

In [ ]:
vote_clf = VotingClassifier(estimators=estimator,voting="hard",n_jobs=-1)
vote_clf.fit(X_train,y_train)
preds=vote_clf.predict(X_test)
acc=accuracy_score(y_test,preds)
print("ACC for Hard  Voting Classifier is",acc)

# 2.Soft Voting Classifier

In [ ]:
vote_clf = VotingClassifier(estimators=estimator,voting="soft")
vote_clf.fit(X_train,y_train)
preds=vote_clf.predict(X_test)
acc=accuracy_score(y_test,preds)
print("ACC for  Soft Voting Classifier is",acc)

# Bagging

In [ ]:
estimator=[("LR",Lasso(alpha=0.1)),
           ("GBC",GradientBoostingClassifier()),
           ("Basiyian",BayesianRidge()),
          ]

In [ ]:
for name,clf in estimators:
    clf1 = BaggingClassifier(clf,,max_samples=0.4)
    bagging_scores = cross_val_score(clf1,X_train,y_train,cv=2,n_jobs=-1)
    print("std",bagging_scores.mean()," bagging ",bagging_scores.std())

# Boosting

In [ ]:
adabost = AdaBoostClassifier(random_state=1)
adaboost.fit(X_train,y_train)
adaboost.score(X_test,y_test)